# TRATAMIENTO DE DATOS CON PYSPARK

In [1]:
# 1. CONFIGURACIÓN E IMPORTACIONES

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

# Iniciamos Spark
spark = SparkSession.builder \
    .appName("FinPlus_ETL_Limpieza") \
    .getOrCreate()

# Rutas de los datos
DATA_PATH = "/home/jovyan/work/data/"  
OUTPUT_PATH = "/home/jovyan/work/data/curated/"

In [2]:
# 2. INGESTA DE DATOS 

# Cargar CLIENTS.csv
# Usamos header=True para leer la cabecera e inferSchema=True para detectar números
df_clients = spark.read.csv(DATA_PATH + "CLIENTS.csv", header=True, inferSchema=True, sep=',')

# Cargar BEHAVIOURAL.parquet
df_behav = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")

# Verificamos qué columnas tenemos 
print("Esquema Clients:")
df_clients.printSchema()

print("Esquema Behavioural:")
df_behav.printSchema()

Esquema Clients:
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = true)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- CAR_AGE: double (nullable = true)
 |-- FAMILY_SIZE: double (nullable = true)
 |-- REACTIVE_SCORING: double (nullable = true)
 |-- PROACTIVE_SCORING: double (nullable = true)
 |-- BEHAVIORAL_SCORING: double (nullable = true)
 |-- DAYS_LAST_INFO_CHANGE: d

Primero analizamos las variables de tipo string. Hay de distintos tipos:

- Categóricas puras: "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY" (en ambas tablas).

- Categóricas numéricas: "NON_COMPLIANT_CONTRACT", "DIGITAL_CLIENT".

- Fecha: "DATE" (sólo en behavioural).

- Identificadores: "CLIENT_ID" (en ambas tablas), "CONTRACT_ID".

Analizamos las filas duplicadas

In [3]:
def auditar_duplicados_completo(df, nombre_tabla):
    print(f"VERIFICACIÓN DE DUPLICADOS: {nombre_tabla}\n ")
    
    # 1. Cálculos Básicos
    total_rows = df.count()
    distinct_rows = df.distinct().count()
    num_duplicados = total_rows - distinct_rows
    
    print(f"• Total filas:      {total_rows}")
    print(f"• Filas únicas:     {distinct_rows}")
    print(f"• Duplicados:       {num_duplicados}")
    
    # 2. Lógica Condicional
    if num_duplicados > 0:
        pct = (num_duplicados / total_rows) * 100
        print(f"\n AVISO: Hay {num_duplicados} filas repetidas ({pct:.2f}%).")
        print("   Mostrando ejemplos de filas idénticas:")
        
        # Esta parte solo se ejecuta si hay duplicados 
        # Agrupamos por TODAS las columnas para encontrar filas 100% idénticas
        (df.groupBy(df.columns)
           .count()
           .where(F.col("count") > 1)
           .orderBy(F.col("count").desc()) # Ponemos las más repetidas arriba
           .show(5, truncate=False))
        
    else:
        print("\nLimpio. No existen filas duplicadas exactas.")
        

# --- EJECUCIÓN ---
auditar_duplicados_completo(df_clients, "CLIENTS")
auditar_duplicados_completo(df_behav, "BEHAVIOURAL")

VERIFICACIÓN DE DUPLICADOS: CLIENTS
 
• Total filas:      162977
• Filas únicas:     162977
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.
VERIFICACIÓN DE DUPLICADOS: BEHAVIOURAL
 
• Total filas:      1724854
• Filas únicas:     1724854
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.


Ahora analizamos los posibles IDs duplicados

In [4]:
def auditar_clave_primaria(df, col_id, nombre_tabla):
    print(f"VERIFICACIÓN DE CLAVE ÚNICA ({col_id}): {nombre_tabla}\n ")
    
    # 1. Contamos filas totales
    total = df.count()
    
    # 2. Contamos IDs únicos
    unicos = df.select(col_id).distinct().count()
    
    # 3. Diferencia
    dif = total - unicos
    
    if dif > 0:
        print(f"AVISO: Hay {dif} IDs repetidos que NO son filas idénticas.")
        print("   Esto significa que tienes clientes con datos conflictivos.")
        print("   Ejemplo de IDs repetidos:")
        
        # Mostramos cuáles son los culpables
        (df.groupBy(col_id)
           .count()
           .where(F.col("count") > 1)
           .show(5))
    else:
        print(f"CORRECTO: La columna {col_id} es una clave primaria única.")

# Ejecutamos solo para CLIENTS
auditar_clave_primaria(df_clients, "CLIENT_ID", "CLIENTS")

VERIFICACIÓN DE CLAVE ÚNICA (CLIENT_ID): CLIENTS
 
CORRECTO: La columna CLIENT_ID es una clave primaria única.


Como vemos que no hay duplicados, no es necesario hacer limpieza de estos. Nos enfocaremos en los NaNs.

In [5]:
# 1. Definimos la función mejorada (le añadimos un título para que quede claro)
def inspeccionar_datos(df, nombre_tabla):
    
    print(f"INSPECCIÓN DE: {nombre_tabla}\n")
    
    print(f"Dimensiones: {df.count()} filas x {len(df.columns)} columnas")
    
    print(f"\n--- 1. Conteo de Nulos ---")
    # Calculamos nulos
    exprs_nulos = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]
    # Mostramos verticalmente para leer mejor
    df.agg(*exprs_nulos).show(vertical=True, truncate=False)

# 2. Ejecutamos la inspección (SIN meterlo dentro de otro print)
inspeccionar_datos(df_clients, "CLIENTS")
inspeccionar_datos(df_behav, "BEHAVIOURAL")

INSPECCIÓN DE: CLIENTS

Dimensiones: 162977 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0-----------------------------
 CLIENT_ID                   | 0      
 NON_COMPLIANT_CONTRACT      | 0      
 NAME_PRODUCT_TYPE           | 0      
 GENDER                      | 0      
 TOTAL_INCOME                | 0      
 AMOUNT_PRODUCT              | 0      
 INSTALLMENT                 | 7      
 EDUCATION                   | 39640  
 MARITAL_STATUS              | 2      
 HOME_SITUATION              | 0      
 REGION_SCORE                | 0      
 AGE_IN_YEARS                | 0      
 JOB_SENIORITY               | 29174  
 HOME_SENIORITY              | 0      
 LAST_UPDATE                 | 0      
 OWN_INSURANCE_CAR           | 0      
 CAR_AGE                     | 107550 
 FAMILY_SIZE                 | 2      
 REACTIVE_SCORING            | 91901  
 PROACTIVE_SCORING           | 337    
 BEHAVIORAL_SCORING          | 32246  
 DAYS_LAST_INFO_CHANGE       | 1      
 NUMBER_OF_

Se observa que solamente tendremos que tratar nulls de la tabla de clientes. Pero además, no nos interesa eliminar filas porque eliminaríamos clientes, y menos en variables con tantos nulls como CAR_AGE, donde estaríamos eliminando más del 66% de los clientes. La estrategia ganadora en Big Data es "Imputar lo masivo, borrar lo anecdótico".

Primero de todo, eliminaremos a los clientes que no proporcionen casi datos, pues prácticamente es como si no existiese.

In [6]:
# ELIMINAR "CLIENTES ZOMBIE" (Filas con demasiados nulos)

print(f"Filas antes de limpieza fina: {df_clients.count()}")

# Tenemos unas 45 columnas. Si a un cliente le faltan más de 20 datos, no nos sirve.
# thresh=25 significa: "Mantener solo si tiene al menos 25 columnas con datos válidos"
df_clients = df_clients.dropna(thresh=25) 

print(f"Filas tras limpieza fina: {df_clients.count()}")

Filas antes de limpieza fina: 162977
Filas tras limpieza fina: 161078


In [7]:

# 3. LIMPIEZA Y TRANSFORMACIÓN

# A) LIMPIEZA DE STRINGS (CLIENTS)
# Lista de tus columnas categóricas reales (copiadas de tu esquema)
cols_categ_puras = [
    "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY"
]

# Normalizamos: quitamos espacios (trim) y pasamos a mayúsculas o minúsculas
for col_name in cols_categ_puras:
    # Solo si la columna existe en el dataframe
    if col_name in df_clients.columns:
        df_clients = df_clients.withColumn(col_name, F.trim(F.upper(F.col(col_name))))

# B) CONVERSIÓN DE FECHAS (BEHAVIOURAL)
# Tu columna DATE es string, hay que pasarla a formato fecha
# Spark suele ser listo, pero si falla, prueba con formato específico ej: "dd/MM/yyyy"
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))



In [8]:
# C) LIMPIEZA DE NULOS 

# 1. GRUPO "NO APLICA" (Rellenar con -1)
# Variables donde Nulo significa "No tiene" o "No disponible"
# CAR_AGE (66% nulos), JOB_SENIORITY, SCORING...
cols_flag = ["CAR_AGE", "JOB_SENIORITY", "REACTIVE_SCORING", "BEHAVIORAL_SCORING", "PROACTIVE_SCORING"]
df_clients = df_clients.fillna(-1, subset=cols_flag)

# 2. GRUPO "SIN HISTORIAL" (Rellenar con 0)
# El grupo de los 8770 nulos. Si no hay datos de préstamos, asumimos 0.
# Buscamos todas las columnas de préstamos (LOAN_) y estados (NUM_STATUS_)
cols_financieras = [c for c in df_clients.columns if c.startswith("LOAN_") or c.startswith("NUM_")]
# Añadimos otras que tengan sentido ser 0
cols_financieras.extend(["NUM_PREVIOUS_LOAN_APP", "NUMBER_OF_PRODUCTS", "Num_flag_insured"]) # Asegúrate de usar el nombre exacto (mayusc/minusc)

# Filtramos solo las que existen en el DF para no dar error
cols_financieras = [c for c in cols_financieras if c in df_clients.columns]
df_clients = df_clients.fillna(0, subset=cols_financieras)

# 3. GRUPO "CATEGÓRICO DESCONOCIDO" (Rellenar con 'Unknown')
# EDUCATION tiene 39k nulos. No podemos inventárnosla.
cols_categ_nulos = ["EDUCATION", "EMPLOYER_ORGANIZATION_TYPE"]
df_clients = df_clients.fillna("UNKNOWN", subset=cols_categ_nulos)

# 4. GRUPO "ANECDÓTICO" (Estrategia: Salvar al Cliente)
# Al ser poquísimos nulos, preferimos imputar para no perder la ficha del cliente.

# A) Para Numéricas (Installment): Usamos la MEDIANA (más robusta que la media)
# Calculamos la mediana aproximada (approxQuantile es muy eficiente en Spark)
mediana_inst = df_clients.stat.approxQuantile("INSTALLMENT", [0.5], 0.01)[0]
df_clients = df_clients.fillna(mediana_inst, subset=["INSTALLMENT"])

# B) Para Categóricas (Marital Status, Family Size): Usamos "UNKNOWN"
# Usar la moda requeriría más código, y para 2 filas no merece la pena el coste computacional.
# "UNKNOWN" nos permite ver en el Dashboard si hay errores de calidad.
cols_anecdoticas = ["MARITAL_STATUS", "FAMILY_SIZE"]
df_clients = df_clients.fillna("UNKNOWN", subset=cols_anecdoticas)

print("Limpieza completada.")

Limpieza completada.


In [9]:
inspeccionar_datos(df_clients, "CLIENTS")


INSPECCIÓN DE: CLIENTS

Dimensiones: 161078 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0--------------------------
 CLIENT_ID                   | 0   
 NON_COMPLIANT_CONTRACT      | 0   
 NAME_PRODUCT_TYPE           | 0   
 GENDER                      | 0   
 TOTAL_INCOME                | 0   
 AMOUNT_PRODUCT              | 0   
 INSTALLMENT                 | 0   
 EDUCATION                   | 0   
 MARITAL_STATUS              | 0   
 HOME_SITUATION              | 0   
 REGION_SCORE                | 0   
 AGE_IN_YEARS                | 0   
 JOB_SENIORITY               | 0   
 HOME_SENIORITY              | 0   
 LAST_UPDATE                 | 0   
 OWN_INSURANCE_CAR           | 0   
 CAR_AGE                     | 0   
 FAMILY_SIZE                 | 0   
 REACTIVE_SCORING            | 0   
 PROACTIVE_SCORING           | 0   
 BEHAVIORAL_SCORING          | 0   
 DAYS_LAST_INFO_CHANGE       | 0   
 NUMBER_OF_PRODUCTS          | 0   
 OCCUPATION                  | 0   
 DIGITAL

In [10]:

# 5. INTEGRACIÓN (JOIN) - MODO NOMBRES ORIGINALES


print(" FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)\n")

col_id = "CLIENT_ID" 

# --- A. SNAPSHOT (Último dato por cliente) ---
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))

# Ventana para quedarnos con el último registro
w = Window.partitionBy(col_id).orderBy(F.desc("DATE"))
df_behav_dedup = df_behav.withColumn("rank", F.row_number().over(w)) \
                         .filter(F.col("rank") == 1) \
                         .drop("rank")

# --- B. JOIN ---
print(f"Uniendo tablas por {col_id}...")
df_master = df_clients.join(df_behav_dedup, on=col_id, how="left")

# --- C. INTEGRIDAD ---
filas_clientes = df_clients.count()
filas_master = df_master.count()

if filas_master > filas_clientes:
    diff = filas_master - filas_clientes
    print(f"AVISO: Aún hay {diff} duplicados.")
else:
    print(f"CORRECTO: 1 Cliente = 1 Fila.")

# --- D. LIMPIEZA POST-JOIN (Nulos a 0) ---
# Tras el join, las columnas financieras de BEHAVIOURAL pueden tener nulos
cols_financieras = [
    "CREDICT_CARD_BALANCE", "CREDIT_CARD_LIMIT", "NUMBER_DRAWINGS", 
    "CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", 
    "CREDIT_CARD_DRAWINGS_OTHER", "CREDIT_CARD_DRAWINGS", "CREDIT_CARD_PAYMENT"
]

cols_existentes = [c for c in cols_financieras if c in df_master.columns]
df_master = df_master.fillna(0, subset=cols_existentes)

# E. INFORMACIÓN DE LA TABLA 
print(" ESTADO ACTUAL (PRE-REDUCCIÓN)\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print("\nEsquema Actual ")
df_master.printSchema()

 FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)

Uniendo tablas por CLIENT_ID...
CORRECTO: 1 Cliente = 1 Fila.
 ESTADO ACTUAL (PRE-REDUCCIÓN)

Dimensiones: 161078 filas x 58 columnas

Esquema Actual 
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = false)
 |-- EDUCATION: string (nullable = false)
 |-- MARITAL_STATUS: string (nullable = false)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = false)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- CAR_AGE: double (nullable = false)
 |-- FAMILY_SIZE: double (null

In [11]:

# 6. FEATURE ENGINEERING Y REDUCCIÓN FINAL

print("FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL\n ")

# --- A. CREACIÓN DE KPIs (Información de Negocio) ---
print("Generando KPIs estratégicos...")

# 1. KPI Gasto Total Tarjeta (Suma de los parciales)
df_master = df_master.withColumn(
    "KPI_TOTAL_SPEND",
    F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER")
)

# 2. KPI Ratio Endeudamiento (Deuda / Ingresos)
# Sumamos 1 al ingreso para evitar divisiones por cero
df_master = df_master.withColumn(
    "KPI_DEBT_RATIO",
    F.round(F.col("CREDICT_CARD_BALANCE") / (F.col("TOTAL_INCOME") + 1), 4)
)

# 3. KPI Grupo de Edad (Simplificación demográfica)
df_master = df_master.withColumn(
    "KPI_AGE_GROUP",
    F.when(F.col("AGE_IN_YEARS") < 30, "Joven")
     .when(F.col("AGE_IN_YEARS") < 50, "Adulto")
     .otherwise("Senior")
)

# 4. KPI Volatilidad de Préstamos (Max - Min)
df_master = df_master.withColumn(
    "KPI_LOAN_VOLATILITY",
    F.col("LOAN_CREDIT_GRANTED_MAX") - F.col("LOAN_CREDIT_GRANTED_MIN")
)

# 5. KPI Ratio de Aprobación (Lo que le dieron / Lo que pidió)
# Indica la confianza del banco en el cliente
df_master = df_master.withColumn(
    "KPI_APPROVAL_RATIO",
    F.round(F.col("LOAN_CREDIT_GRANTED_SUM") / (F.col("LOAN_APPLICATION_AMOUNT_SUM") + 1), 2)
)

# 6. KPI Tasa de Rechazo (Solicitudes denegadas / Total)
# Resume los estados conflictivos
df_master = df_master.withColumn(
    "KPI_DENIAL_RATE",
    F.round(F.col("NUM_STATUS_DENIED") / 
            (F.col("NUM_STATUS_AUTHORIZED") + F.col("NUM_STATUS_DENIED") + F.col("NUM_STATUS_ANNULLED") + 1), 2)
)

# --- B. REDUCCIÓN DE COLUMNAS (Limpieza de "Grasa") ---
print("Eliminando columnas redundantes e inútiles...")

cols_a_borrar = [
    # 1. Redundantes de Tarjeta (Sustituidas por KPI_TOTAL_SPEND)
    "CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", "CREDIT_CARD_DRAWINGS_OTHER", 
    "CREDIT_CARD_DRAWINGS", # Dato duplicado original
    
    # 2. Redundantes de Préstamos (Sustituidas por KPI_LOAN_VOLATILITY y APPROVAL)
    "LOAN_ANNUITY_PAYMENT_MAX", "LOAN_ANNUITY_PAYMENT_MIN",
    "LOAN_APPLICATION_AMOUNT_MAX", "LOAN_APPLICATION_AMOUNT_MIN",
    "LOAN_CREDIT_GRANTED_MAX", "LOAN_CREDIT_GRANTED_MIN",
    "LOAN_VARIABLE_RATE_MAX", "LOAN_VARIABLE_RATE_MIN",

    # 3. Basura Técnica y Duplicados
    "CONTRACT_ID",  # ID interno técnico (inútil para negocio)
    "CURRENCY"      # Columna duplicada por el Join (borramos ambas)
    ]

# Borrado seguro (solo las que existan)
cols_finales_borrar = [c for c in cols_a_borrar if c in df_master.columns]
df_master = df_master.drop(*cols_finales_borrar)

# --- C. DIMENSIONES FINALES ---

print("ESTADO FINAL OPTIMIZADO\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print(f"(Se han eliminado {len(cols_finales_borrar)} columnas redundantes y creado 6 KPIs estratégicos)")

# --- D. GUARDADO FINAL ---
ruta_final = OUTPUT_PATH + "Master_FinPlus.parquet"
df_master.write.mode("overwrite").parquet(ruta_final)
print(f"\nGuardado archivo en: {ruta_final}.")

# --- E. GUARDADO EN PARQUET ÚNICO ---

df_pandas = df_master.toPandas()

nombre_archivo = "Master_FinPlus_Final.parquet"
ruta_completa = OUTPUT_PATH + nombre_archivo

# Necesitas tener instalada la librería pyarrow o fastparquet (suele venir en Docker)
df_pandas.to_parquet(ruta_completa, index=False)

print(f"\nGuardado.")
print(f" Ruta: {ruta_completa}")

FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL
 
Generando KPIs estratégicos...
Eliminando columnas redundantes e inútiles...
ESTADO FINAL OPTIMIZADO

Dimensiones: 161078 filas x 49 columnas
(Se han eliminado 14 columnas redundantes y creado 6 KPIs estratégicos)

Guardado archivo en: /home/jovyan/work/data/curated/Master_FinPlus.parquet.

Guardado.
 Ruta: /home/jovyan/work/data/curated/Master_FinPlus_Final.parquet


In [12]:
def auditar_outliers(df, cols_numericas):
    
    print(f" VERIFICACIÓN DE OUTLIERS (Método IQR)")
    
    
    for col in cols_numericas:
        # 1. Calculamos Cuartiles (25% y 75%)
        quantiles = df.stat.approxQuantile(col, [0.25, 0.75], 0.01)
        q1, q3 = quantiles[0], quantiles[1]
        iqr = q3 - q1
        
        # 2. Definimos límites (Bigotes del Boxplot)
        limite_inf = q1 - 1.5 * iqr
        limite_sup = q3 + 1.5 * iqr
        
        # 3. Contamos cuántos se salen
        outliers = df.filter((F.col(col) < limite_inf) | (F.col(col) > limite_sup))
        num_outliers = outliers.count()
        
        if num_outliers > 0:
            print(f"\n Columna: {col}")
            print(f"   Rango Normal: [{limite_inf:.2f}  a  {limite_sup:.2f}]")
            print(f"   Outliers detectados: {num_outliers} filas")
            
            # Mostramos los valores más extremos para ver si son errores o VIPs
            print(f"   Ejemplos (Top Extremos):")
            outliers.select(col).orderBy(F.desc(col)).show(3)
        else:
            print(f"\n Columna: {col} -> Sin outliers estadísticos.")

# Definimos las columnas numéricas críticas para analizar
cols_analisis = ["TOTAL_INCOME", "AGE_IN_YEARS", "AMOUNT_PRODUCT", "CREDICT_CARD_BALANCE"]

# Ejecutamos
auditar_outliers(df_master, cols_analisis)

 VERIFICACIÓN DE OUTLIERS (Método IQR)

 Columna: TOTAL_INCOME
   Rango Normal: [-270.00  a  4050.00]
   Outliers detectados: 7244 filas
   Ejemplos (Top Extremos):
+------------+
|TOTAL_INCOME|
+------------+
|   1404000.0|
|   216001.08|
|    108000.0|
+------------+
only showing top 3 rows


 Columna: AGE_IN_YEARS -> Sin outliers estadísticos.

 Columna: AMOUNT_PRODUCT
   Rango Normal: [-6455.70  a  19399.50]
   Outliers detectados: 3395 filas
   Ejemplos (Top Extremos):
+--------------+
|AMOUNT_PRODUCT|
+--------------+
|     48486.195|
|     48486.195|
|     48486.195|
+--------------+
only showing top 3 rows


 Columna: CREDICT_CARD_BALANCE
   Rango Normal: [0.00  a  0.00]
   Outliers detectados: 16332 filas
   Ejemplos (Top Extremos):
+--------------------+
|CREDICT_CARD_BALANCE|
+--------------------+
|            14526.13|
|            12122.07|
|            11500.21|
+--------------------+
only showing top 3 rows



# INDICADORES Y ANÁLISIS DE COMPORTAMIENTO

### ACTIVIDAD CLIENTE

- Cálculo de métricas de actividad:
    - Recencia (R)

    - Frecuencia (F)

    - Intensidad (I)

- Ventanas de actividad (30/90/180 días)

- Meses activos

- Clasificación de actividad (Alta / Media / Baja)

- Interpretación clara de negocio


In [18]:
# ==========================================
# 1. CARGAR DATOS YA PROCESADOS
# ==========================================
DATA_PATH = "/home/jovyan/work/data/"

beh = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")
df_master = spark.read.parquet(DATA_PATH + "curated/Master_FinPlus.parquet")

beh = beh.withColumn("DATE", F.to_date("DATE", "yyyy-MM-dd"))
max_date = beh.agg(F.max("DATE")).first()[0]




In [19]:
# ==========================================
# 2. MÉTRICAS DE ACTIVIDAD
# ==========================================

# --- RECENCIA ---
recencia_df = beh.groupBy("CLIENT_ID").agg(
    F.max("DATE").alias("last_activity_date")
).withColumn(
    "RECENCY_DAYS", F.datediff(F.lit(max_date), F.col("last_activity_date"))
)

print("\n================ RECENCIA (R) ================\n")
print("Interpretación: número de días desde la última actividad del cliente.")
recencia_df.describe("RECENCY_DAYS").show()

print("\n--- Top 10 clientes más recientes ---")
recencia_df.orderBy(F.col("RECENCY_DAYS").asc()).show(10, truncate=False)

print("\n--- Top 10 clientes más abandonados ---")
recencia_df.orderBy(F.col("RECENCY_DAYS").desc()).show(10, truncate=False)




================ RECENCIA (R) ================

Interpretación: número de días desde la última actividad del cliente.
+-------+------------------+
|summary|      RECENCY_DAYS|
+-------+------------------+
|  count|             46046|
|   mean|15.444794336098685|
| stddev| 21.54619528266414|
|    min|                 0|
|    max|                93|
+-------+------------------+


--- Top 10 clientes más recientes ---
+------------+------------------+------------+
|CLIENT_ID   |last_activity_date|RECENCY_DAYS|
+------------+------------------+------------+
|ES182303796D|2021-12-31        |0           |
|ES182245752Y|2021-12-31        |0           |
|ES182293250V|2021-12-31        |0           |
|ES182245476M|2021-12-31        |0           |
|ES182112694Y|2021-12-31        |0           |
|ES182189508S|2021-12-31        |0           |
|ES182433571C|2021-12-31        |0           |
|ES182232062V|2021-12-31        |0           |
|ES182378189N|2021-12-31        |0           |
|ES182279031S|20

In [27]:

# --- FRECUENCIA ---
frecuencia_df = beh.groupBy("CLIENT_ID").agg(
    F.count("*").alias("FREQUENCY_COUNT")
)

print("\n================ FRECUENCIA (F) ================\n")
print("Interpretación: cuántos movimientos totales ha realizado el cliente.")
frecuencia_df.describe("FREQUENCY_COUNT").show()

print("\n--- Top 10 clientes con mayor frecuencia ---")
frecuencia_df.orderBy(F.col("FREQUENCY_COUNT").desc()).show(10, truncate=False)


================ FRECUENCIA (F) ================

Interpretación: cuántos movimientos totales ha realizado el cliente.
+-------+-----------------+
|summary|  FREQUENCY_COUNT|
+-------+-----------------+
|  count|            46046|
|   mean|37.45936672023628|
| stddev|33.78619079695795|
|    min|                1|
|    max|              192|
+-------+-----------------+


--- Top 10 clientes con mayor frecuencia ---
+------------+---------------+
|CLIENT_ID   |FREQUENCY_COUNT|
+------------+---------------+
|ES182186401T|192            |
|ES182128827G|129            |
|ES182192917N|126            |
|ES182283225D|122            |
|ES182378495D|122            |
|ES182155668A|121            |
|ES182253915Y|121            |
|ES182146380G|121            |
|ES182267366X|120            |
|ES182210848C|120            |
+------------+---------------+
only showing top 10 rows



In [21]:
# --- INTENSIDAD ---
beh = beh.withColumn(
    "KPI_TOTAL_SPEND",
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_ATM"), F.lit(0)) +
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_POS"), F.lit(0)) +
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_OTHER"), F.lit(0))
)

intensidad_df = beh.groupBy("CLIENT_ID").agg(
    F.avg("KPI_TOTAL_SPEND").alias("INTENSITY_AVG_SPEND")
)

print("\n================ INTENSIDAD (I) ================\n")
print("Interpretación: gasto promedio por movimiento del cliente.")
intensidad_df.describe("INTENSITY_AVG_SPEND").show()

print("\n--- Top 10 clientes con mayor intensidad ---")
intensidad_df.orderBy(F.col("INTENSITY_AVG_SPEND").desc()).show(10, truncate=False)




================ INTENSIDAD (I) ================

Interpretación: gasto promedio por movimiento del cliente.
+-------+-------------------+
|summary|INTENSITY_AVG_SPEND|
+-------+-------------------+
|  count|              46046|
|   mean| 165.34456214658363|
| stddev|  310.5056279798483|
|    min|                0.0|
|    max|  9629.483333333334|
+-------+-------------------+


--- Top 10 clientes con mayor intensidad ---
+------------+-------------------+
|CLIENT_ID   |INTENSITY_AVG_SPEND|
+------------+-------------------+
|ES182227107Z|9629.483333333334  |
|ES182354235A|8642.13            |
|ES182436756T|7395.070000000001  |
|ES182156500S|5901.773846153846  |
|ES182366160A|5899.592142857144  |
|ES182390308Q|5731.825           |
|ES182247132U|5427.0             |
|ES182396618L|5346.0             |
|ES182129030R|5245.332173913043  |
|ES182250483A|5186.4158333333335 |
+------------+-------------------+
only showing top 10 rows



In [22]:

# ==========================================
# 3. ACTIVIDAD 30 / 90 / 180 DÍAS
# ==========================================
beh_windows = beh.withColumn(
    "DAYS_FROM_REF", F.datediff(F.lit(max_date), F.col("DATE"))
)

ventanas_df = beh_windows.groupBy("CLIENT_ID").agg(
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 30, 1).otherwise(0)).alias("ACTIVITY_30D"),
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 90, 1).otherwise(0)).alias("ACTIVITY_90D"),
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 180, 1).otherwise(0)).alias("ACTIVITY_180D")
)

print("\n================ ACTIVIDAD EN VENTANAS ================\n")
print("Interpretación: cuántas interacciones ha tenido el cliente en los últimos X días.\n")

print("--- Top 10 actividad últimos 30 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_30D").desc()).show(10, truncate=False)

print("--- Top 10 actividad últimos 90 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_90D").desc()).show(10, truncate=False)

print("--- Top 10 actividad últimos 180 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_180D").desc()).show(10, truncate=False)




================ ACTIVIDAD EN VENTANAS ================

Interpretación: cuántas interacciones ha tenido el cliente en los últimos X días.

--- Top 10 actividad últimos 30 días ---
+------------+------------+------------+-------------+
|CLIENT_ID   |ACTIVITY_30D|ACTIVITY_90D|ACTIVITY_180D|
+------------+------------+------------+-------------+
|ES182403907W|2           |6           |12           |
|ES182334710B|2           |6           |12           |
|ES182210848C|2           |6           |12           |
|ES182347896Q|2           |6           |12           |
|ES182243027C|2           |6           |11           |
|ES182407401Q|2           |6           |12           |
|ES182127891W|2           |5           |8            |
|ES182150696Z|2           |6           |12           |
|ES182100594F|2           |4           |7            |
|ES182283225D|2           |6           |12           |
+------------+------------+------------+-------------+
only showing top 10 rows

--- Top 10 actividad ú

In [23]:

# ==========================================
# 4. MESES ACTIVOS
# ==========================================
beh_month = beh.withColumn(
    "YEAR_MONTH", F.date_format("DATE", "yyyy-MM")
)

meses_activos_df = beh_month.groupBy("CLIENT_ID").agg(
    F.countDistinct("YEAR_MONTH").alias("ACTIVE_MONTHS")
)

print("\n================ MESES ACTIVOS ================\n")
meses_activos_df.orderBy(F.col("ACTIVE_MONTHS").desc()).show(10, truncate=False)




================ MESES ACTIVOS ================

+------------+-------------+
|CLIENT_ID   |ACTIVE_MONTHS|
+------------+-------------+
|ES182319130A|96           |
|ES182173222Z|96           |
|ES182384509S|96           |
|ES182167286W|96           |
|ES182243311L|96           |
|ES182233737B|96           |
|ES182254666P|96           |
|ES182258895P|96           |
|ES182454817C|96           |
|ES182350305O|96           |
+------------+-------------+
only showing top 10 rows



In [24]:

# ==========================================
# 5. COMBINAR TODAS LAS MÉTRICAS
# ==========================================
activity_metrics = (
    recencia_df
    .join(frecuencia_df, "CLIENT_ID", "left")
    .join(intensidad_df, "CLIENT_ID", "left")
    .join(ventanas_df, "CLIENT_ID", "left")
    .join(meses_activos_df, "CLIENT_ID", "left")
)


In [25]:

# ==========================================
# 6. SEGMENTACIÓN DE ACTIVIDAD
# ==========================================
activity_metrics = activity_metrics.withColumn(
    "ACTIVITY_SEGMENT",
    F.when((F.col("RECENCY_DAYS") <= 30) & (F.col("FREQUENCY_COUNT") > 5), "Alta")
     .when((F.col("RECENCY_DAYS") <= 90) & (F.col("FREQUENCY_COUNT") > 2), "Media")
     .otherwise("Baja")
)

print("\n================ SEGMENTACIÓN FINAL ================\n")
activity_metrics.groupBy("ACTIVITY_SEGMENT").count().show()

print("\n--- Muestra de clientes segmentados ---")
activity_metrics.select(
    "CLIENT_ID", "RECENCY_DAYS", "FREQUENCY_COUNT", 
    "INTENSITY_AVG_SPEND", "ACTIVITY_SEGMENT"
).orderBy("ACTIVITY_SEGMENT").show(20, truncate=False)




================ SEGMENTACIÓN FINAL ================

+----------------+-----+
|ACTIVITY_SEGMENT|count|
+----------------+-----+
|            Alta|26002|
|           Media|18448|
|            Baja| 1596|
+----------------+-----+


--- Muestra de clientes segmentados ---
+------------+------------+---------------+-------------------+----------------+
|CLIENT_ID   |RECENCY_DAYS|FREQUENCY_COUNT|INTENSITY_AVG_SPEND|ACTIVITY_SEGMENT|
+------------+------------+---------------+-------------------+----------------+
|ES182222478Q|0           |19             |588.2236842105264  |Alta            |
|ES182325278Y|0           |9              |625.4711111111111  |Alta            |
|ES182133642C|0           |85             |31.129411764705882 |Alta            |
|ES182245476M|0           |10             |0.0                |Alta            |
|ES182413873B|0           |91             |106.26186813186811 |Alta            |
|ES182245752Y|0           |26             |0.0                |Alta            |

In [26]:

# ==========================================
# 7. UNIR A MASTER FINAL
# ==========================================
df_final = df_master.join(activity_metrics, "CLIENT_ID", "left")
df_final.write.mode("overwrite").parquet(DATA_PATH + "Master_FinPlus_Activity.parquet")

print("\nArchivo guardado correctamente en: Master_FinPlus_Activity.parquet")



Archivo guardado correctamente en: Master_FinPlus_Activity.parquet


### VALOR ECONÓMICO

In [ ]:
import os

DATA_PATH = "/home/jovyan/work/data/"
OUT_PATH = "/home/jovyan/work/data/activity/"  # carpeta de salida de activity/metrics
os.makedirs(OUT_PATH, exist_ok=True)

# 1) Cargar BEHAVIOURAL y MASTER si necesitas KPIs demográficos
beh = spark.read.parquet(os.path.join(DATA_PATH, "BEHAVIOURAL.parquet"))
# Aseguramos formato fecha
beh = beh.withColumn("DATE", F.to_date("DATE", "yyyy-MM-dd"))

# Master (opcional) para incorporar totales / ingresos si quieres
master_path = os.path.join(DATA_PATH, "curated", "Master_FinPlus.parquet")
master = None
if os.path.exists(master_path):
    master = spark.read.parquet(master_path)

print("Loaded BEHAVIOURAL rows:", beh.count(), "  (master found?)", master is not None)

# 2) Normalizar columnas de monto / crear KPI_TOTAL_SPEND si no existe
# (usa mismas columnas que definiste en TRATAMIENTO DE DATOS)
beh = beh.withColumn("CREDIT_CARD_DRAWINGS_ATM", F.coalesce(F.col("CREDIT_CARD_DRAWINGS_ATM"), F.lit(0.0))) \
         .withColumn("CREDIT_CARD_DRAWINGS_POS", F.coalesce(F.col("CREDIT_CARD_DRAWINGS_POS"), F.lit(0.0))) \
         .withColumn("CREDIT_CARD_DRAWINGS_OTHER", F.coalesce(F.col("CREDIT_CARD_DRAWINGS_OTHER"), F.lit(0.0))) \
         .withColumn("CREDIT_CARD_DRAWINGS", F.coalesce(F.col("CREDIT_CARD_DRAWINGS"), F.lit(0.0))) \
         .withColumn("CREDIT_CARD_PAYMENT", F.coalesce(F.col("CREDIT_CARD_PAYMENT"), F.lit(0.0))) \
         .withColumn("CREDICT_CARD_BALANCE", F.coalesce(F.col("CREDICT_CARD_BALANCE"), F.lit(0.0))) \
         .withColumn("CREDIT_CARD_LIMIT", F.coalesce(F.col("CREDIT_CARD_LIMIT"), F.lit(0.0)))

beh = beh.withColumn("KPI_TOTAL_SPEND",
    F.when(F.col("CREDIT_CARD_DRAWINGS") > 0, F.col("CREDIT_CARD_DRAWINGS"))
     .otherwise(F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_OTHER"))
)

# Flag actividad (por fila)
beh = beh.withColumn("HAS_ACTIVITY", F.when(F.col("KPI_TOTAL_SPEND") > 0, 1).otherwise(0))

# 3) Agregados por cliente: monetary_total, payments_total, avg_balance, avg_limit, utilization
agg_econ = beh.groupBy("CLIENT_ID").agg(
    F.sum("KPI_TOTAL_SPEND").alias("MONETARY_TOTAL"),         # volumen total observado
    F.sum("CREDIT_CARD_PAYMENT").alias("TOTAL_PAYMENTS"),    # pagos realizados (proxy recuperación)
    F.avg("CREDICT_CARD_BALANCE").alias("AVG_BALANCE"),      # balance medio
    F.avg("CREDIT_CARD_LIMIT").alias("AVG_LIMIT"),           # límite medio
    F.count(F.when(F.col("HAS_ACTIVITY")==1, True)).alias("TX_COUNT"),
    F.min("DATE").alias("FIRST_TX_DATE"),
    F.max("DATE").alias("LAST_TX_DATE")
)

# Tenure en meses (si FIRST_TX_DATE existe)
agg_econ = agg_econ.withColumn("TENURE_MONTHS", 
                               F.when(F.col("FIRST_TX_DATE").isNotNull(), 
                                      F.floor(F.months_between(F.col("LAST_TX_DATE"), F.col("FIRST_TX_DATE"))).cast("int") + 1)
                                .otherwise(F.lit(1))
                              )

# Ratios: utilization (balance/limit) y payment rate (payments / spend)
agg_econ = agg_econ.withColumn("UTILIZATION", F.when(F.col("AVG_LIMIT")>0, F.col("AVG_BALANCE")/F.col("AVG_LIMIT")).otherwise(F.lit(0.0))) \
                   .withColumn("PAYMENT_RATE", F.when(F.col("MONETARY_TOTAL")>0, F.col("TOTAL_PAYMENTS")/F.col("MONETARY_TOTAL")).otherwise(F.lit(0.0))) \
                   .withColumn("ARPU_PROXY", F.col("MONETARY_TOTAL"))  # ARPU sobre periodo observado; comentado debajo

# 4) ARPU / mediana / total revenue / top-decile share
# Observed totals (period bounded by BEHAVIOURAL dataset)
total_rev = agg_econ.agg(F.sum("MONETARY_TOTAL").alias("TOTAL_REVENUE")).collect()[0]["TOTAL_REVENUE"]
arpu = agg_econ.agg(F.mean("MONETARY_TOTAL").alias("ARPU")).collect()[0]["ARPU"]
median_mon = agg_econ.approxQuantile("MONETARY_TOTAL", [0.5], 0.01)[0]
p90 = agg_econ.approxQuantile("MONETARY_TOTAL", [0.9], 0.01)[0]
top10_rev = agg_econ.filter(F.col("MONETARY_TOTAL") >= p90).agg(F.sum("MONETARY_TOTAL").alias("TOP10_REV")).collect()[0]["TOP10_REV"]
top10_share = (top10_rev / total_rev) if total_rev and total_rev>0 else None

print("\n=== KPI GLOBAL: Valor Económico ===")
print(f"Total revenue (observed in dataset): {total_rev:.2f}")
print(f"ARPU (mean monetary_total per client): {arpu:.2f}")
print(f"Median monetary_total: {median_mon:.2f}")
print(f"90th percentile (threshold for top10): {p90:.2f}")
print(f"Top10 revenue: {top10_rev:.2f}  (share: {top10_share:.2%} )" if top10_share is not None else "Top10 share unavailable")

# Interpretación:
print("\nInterpretación breve:")
print("- ARPU: media de ingreso por cliente en el periodo observado. OJO: si el periodo cubre varios meses, ARPU es acumulado.")
print("- top10_share alto (>50%) => dependencia alta de pocos clientes; riesgo de concentración.")
print("- PAYMENT_RATE bajo indica clientes que consumen (spend) pero pagan poco en el periodo observado; puede indicar morosidad o timing.")

# 5) Revenues por canal (POS/ATM/OTHER) - monto y % por cliente segment y global
channel = beh.groupBy("CLIENT_ID").agg(
    F.sum("CREDIT_CARD_DRAWINGS_POS").alias("POS_AMOUNT"),
    F.sum("CREDIT_CARD_DRAWINGS_ATM").alias("ATM_AMOUNT"),
    F.sum("CREDIT_CARD_DRAWINGS_OTHER").alias("OTHER_AMOUNT")
).withColumn("CHANNEL_TOTAL", F.col("POS_AMOUNT")+F.col("ATM_AMOUNT")+F.col("OTHER_AMOUNT"))

# Global shares
global_channels = channel.agg(
    F.sum("POS_AMOUNT").alias("POS_TOTAL"),
    F.sum("ATM_AMOUNT").alias("ATM_TOTAL"),
    F.sum("OTHER_AMOUNT").alias("OTHER_TOTAL")
).collect()[0]
pos_total = global_channels["POS_TOTAL"] or 0.0
atm_total = global_channels["ATM_TOTAL"] or 0.0
other_total = global_channels["OTHER_TOTAL"] or 0.0
grand = pos_total + atm_total + other_total
print(f"\nRevenue by channel (global): POS={pos_total:.2f}, ATM={atm_total:.2f}, OTHER={other_total:.2f}, grand={grand:.2f}")
if grand>0:
    print(f"Shares: POS={pos_total/grand:.2%}, ATM={atm_total/grand:.2%}, OTHER={other_total/grand:.2%}")

# 6) LTV proxy: monetary_total / tenure_months and monetary_total absolute
# (Proxy porque no tenemos margen ni lifetime completo; es un indicador relativo)
agg_econ = agg_econ.withColumn("LTV_PROXY_MONTHLY", F.when(F.col("TENURE_MONTHS")>0, F.col("MONETARY_TOTAL")/F.col("TENURE_MONTHS")).otherwise(F.col("MONETARY_TOTAL")))

# 7) High-value but low-activity identification (oportunidades)
# Definimos high value como > 75th percentile monetary, low activity as TX_COUNT below median
mon75 = agg_econ.approxQuantile("MONETARY_TOTAL", [0.75], 0.01)[0]
tx_med = agg_econ.approxQuantile("TX_COUNT", [0.5], 0.01)[0]

hv_la = agg_econ.filter((F.col("MONETARY_TOTAL") > mon75) & (F.col("TX_COUNT") < tx_med))

print(f"\nHigh-value threshold (75th pct): {mon75:.2f}, TX_COUNT median: {tx_med}")
print("Ejemplos High-Value & Low-Activity (top 10 por monetary):")
hv_la.orderBy(F.col("MONETARY_TOTAL").desc()).select("CLIENT_ID","MONETARY_TOTAL","TX_COUNT","TENURE_MONTHS","LTV_PROXY_MONTHLY").show(10, truncate=False)

# 8) Cohorte por primer mes de actividad -> revenue acumulado por cohorte
cohort = beh.groupBy("CLIENT_ID").agg(F.min("DATE").alias("first_tx_date"))
cohort = cohort.withColumn("cohort_month", F.date_format(F.col("first_tx_date"), "yyyy-MM"))

rev_by_cohort = beh.withColumn("cohort_month", F.date_format(F.col("DATE"), "yyyy-MM")) \
                   .join(cohort.select("CLIENT_ID","cohort_month").distinct(), on="CLIENT_ID", how="left") \
                   .groupBy("cohort_month").agg(
                       F.sum("KPI_TOTAL_SPEND").alias("COHORT_REVENUE"),
                       F.countDistinct("CLIENT_ID").alias("COHORT_SIZE")
                   ).orderBy("cohort_month")

print("\nRevenue by cohort_month (sample):")
rev_by_cohort.show(20, truncate=False)

# 9) Clientes con peor PAYMENT_RATE (riesgo) y con mayor MONETARY_TOTAL (impacto)
agg_econ = agg_econ.join(channel.select("CLIENT_ID","POS_AMOUNT","ATM_AMOUNT","OTHER_AMOUNT","CHANNEL_TOTAL"), "CLIENT_ID", "left")
agg_econ = agg_econ.join(beh.groupBy("CLIENT_ID").agg(F.sum("CREDIT_CARD_PAYMENT").alias("TOTAL_PAYMENTS2")), "CLIENT_ID", "left")
agg_econ = agg_econ.withColumn("PAYMENT_RATE2", F.when(F.col("MONETARY_TOTAL")>0, F.col("TOTAL_PAYMENTS2")/F.col("MONETARY_TOTAL")).otherwise(0.0))

print("\nTop 20 clients with low payment rate but high monetary_total (priority risk):")
agg_econ.filter(F.col("PAYMENT_RATE2") < 0.5).orderBy(F.col("MONETARY_TOTAL").desc()).select(
    "CLIENT_ID","MONETARY_TOTAL","TOTAL_PAYMENTS2","PAYMENT_RATE2","TENURE_MONTHS"
).show(20, truncate=False)

# 10) Guardar resultados finales (parquet) y muestra
econ_out = agg_econ.select(
    "CLIENT_ID","MONETARY_TOTAL","TOTAL_PAYMENTS","TOTAL_PAYMENTS2","PAYMENT_RATE","PAYMENT_RATE2",
    "AVG_BALANCE","AVG_LIMIT","UTILIZATION","TX_COUNT","TENURE_MONTHS","LTV_PROXY_MONTHLY",
    "POS_AMOUNT","ATM_AMOUNT","OTHER_AMOUNT","CHANNEL_TOTAL"
).dropDuplicates(["CLIENT_ID"])

econ_out.repartition(50).write.mode("overwrite").parquet(os.path.join(OUT_PATH, "economic_metrics.parquet"))

print("\nEconomic metrics saved to:", os.path.join(OUT_PATH, "economic_metrics.parquet"))

# 11) Resumen ejecutivo impreso (acciones sugeridas)
print("\n=== Resumen ejecutivo - acciones sugeridas ===")
print("- Priorizar recuperación de clientes con PAYMENT_RATE bajo y MONETARY_TOTAL alto (ver tabla anterior).")
print("- Analizar concentración: si top10_share > 40% -> diversificar fuentes de revenue.")
print("- Clientes high-value but low-activity -> ofertas de engagement personalizadas (cross-sell).")
print("- Revisar UTILIZATION elevada (>0.8) -> riesgo de sobre-endeudamiento; monitorizar límites y alertas.")


### INTERACCIÓN Y FIDELIDAD

### RIESGO POTENCIAL

### OPORTUNIDADES COMERCIALES

### ANÁLISIS DE CAUSALIDAD / UPLIFT (para identificar qué ofertas realmente causan más retención)

### EMBEDDING DE COMPORTAMIENTO - SEQUENCE MODELING - (para recomendar productos RNNs / transformers si hay consecuencias largas)

### ANOMALÍA TRANSACCIONAL (para detectar fraudes o glitches del sistema)